In [1]:
import torch
from onnx2torch import convert
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\PC\Documents\projects\carPlates\data\letters420\letter_labels.csv")
lst = []
for i in range(len(df)):
    if df['label'][i] == '0':
        lst.append(i)
df = df.drop(lst, axis = 0)
s = list(set(df['label']))
s = sorted(s)
dic = {i:s[i] for i in range(len(s))}

In [3]:
plates_model_dir = "trained_weights/plates/plates320_s.pt"
letters_model_dir = "trained_weights/letters/letters320_n.pt"
#letters_model_dir = r"C:\Users\PC\Documents\projects\carPlates\yolov5\runs\train\exp35\weights\best.pt"
plates_model = torch.hub.load('ultralytics/yolov5', 'custom', path=plates_model_dir, force_reload=True)
letters_model = torch.hub.load('ultralytics/yolov5', 'custom', path=letters_model_dir, force_reload=True)
# Path to ONNX model
onnx_model_path = r"C:\Users\PC\Documents\projects\carPlates\yolov5\trained_weights\classification\letter_model_mod_1.onnx"
# You can pass the path to the onnx model to convert it or...
class_model = convert(onnx_model_path)


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\PC/.cache\torch\hub\master.zip
YOLOv5  2022-7-13 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\PC/.cache\torch\hub\master.zip
YOLOv5  2022-7-13 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


In [4]:
train_path = r"E:\DataSets\carPlatesDataset\cars_dataset\images\train"
raw_path = r"E:\DataSets\carPlatesDataset\raw"

train_imgs = os.listdir(train_path)
raw_imgs = os.listdir(raw_path)
test_imgs = []
for img in raw_imgs:
    if img not in train_imgs:
        test_imgs.append(img)

test_dirs = [os.path.join(raw_path, img) for img in test_imgs]


In [5]:
img_dir = test_dirs[1]
t1 = time.time()
img = Image.open(img_dir)
plates_pred = plates_model(img)
print(plates_pred.xyxy)
m = torch.Tensor.cpu(plates_pred.xyxy[0])
if m[0][4] < 0.5:
    print("yes")
m = np.array(m, dtype = np.int64)
img = np.array(img, dtype = np.uint8)
plate = Image.fromarray(img[m[1][1]:m[1][3], m[1][0]:m[1][2]])
plate.show()
print(m)
#plate = Image.fromarray(img[int(m[1][0]): int(m[1][2]), int(m[1][1]):int(m[1][3])])
#plate.show()

[tensor([[2.35624e+03, 6.98847e+02, 2.57528e+03, 8.10439e+02, 9.22770e-01, 0.00000e+00],
        [2.87990e+03, 5.15233e+02, 3.01204e+03, 5.81613e+02, 9.20639e-01, 0.00000e+00],
        [3.37030e+03, 4.28701e+02, 3.47018e+03, 4.84572e+02, 8.85897e-01, 0.00000e+00],
        [1.63678e+03, 4.09884e+02, 1.72014e+03, 4.52904e+02, 8.53340e-01, 0.00000e+00],
        [9.76102e+02, 1.08610e+03, 1.10961e+03, 1.24089e+03, 7.99172e-01, 0.00000e+00],
        [2.12340e+03, 2.75564e+02, 2.16455e+03, 2.98308e+02, 3.29551e-01, 0.00000e+00]], device='cuda:0')]
[[2356  698 2575  810    0    0]
 [2879  515 3012  581    0    0]
 [3370  428 3470  484    0    0]
 [1636  409 1720  452    0    0]
 [ 976 1086 1109 1240    0    0]
 [2123  275 2164  298    0    0]]


In [6]:
def get_bin(img):
    """mx, mn = (img[0][0], img[0][0])
    for row in img:
        rmax = max(row)
        rmin = min(row)
        if rmax > mx:
            mx = rmax
        if rmin < mn:
            mn = rmin
    new_img = []
    mn = int(mn)
    mx = int(mx)
    thresh = (mn + mx)/2"""
    new_img = []
    thresh = 125.5
    for row in img:
        nrow = []
        for pixel in row:
            if pixel >= thresh:
                nrow.append(255)
            else:
                nrow.append(0)
        new_img.append(nrow)
    return np.array(new_img)

def get_bin(img):
    (thresh, im_bw) = cv2.threshold(img, 125, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return im_bw

In [7]:
def put_letters_together(lst):
    output = ""
    for letter in lst:
        output += letter + " "
    return output

In [8]:
def get_rotated_img(img):
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(img_gray, 100, 200, None, 3)
        lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)
        total_theta = []
        if lines is not None:
            for i in range(len(lines)):
                if (abs(lines[i][0][1]) < 0.45 or abs(lines[i][0][1] - np.pi) < 0.45):
                    continue
                else:
                    total_theta.append(lines[i][0][1])
            if len(total_theta) != 0:
                avg_theta = sum([theta for theta in total_theta])/len(total_theta) * (180/np.pi) - 90
            else:
                avg_theta = 0
            (h, w) = img.shape[:2]
            (cX, cY) = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D((cX, cY), avg_theta, 1.0)
            rotated = cv2.warpAffine(img, M, (w, h))
            return rotated
        return img

In [9]:
def get_cropped_img(img, percentage = 0.385):
    return img[int(percentage*len(img[:, 0])):, :]

In [10]:
def get_mod_img(img, thresh = 75):
    thresh = 75
    new_img = img
    pixels = np.where((new_img[:, :, 0] <= thresh), (new_img[:, :, 1] <= thresh), (new_img[:, :, 2] <= thresh))
    blacks = np.where((new_img[:, :, 0] <= thresh) & (new_img[:, :, 1] <= thresh) & (new_img[:, :, 2] <= thresh))
    new_img[blacks] = [0]
    new_img[pixels] = [0]
    return new_img

In [11]:
len(test_dirs)

407

In [12]:
def img_pred(img):
    thresh = 0.5
    t1 = time.time()
    plates_pred = plates_model(img)
    plates = torch.Tensor.cpu(plates_pred.xyxy[0])
    all_plates = []
    final_img = np.copy(img)
    for plate in plates:
        if plate[4] < thresh:
            continue
        plate_reading = []
        plate = np.array(plate, dtype = np.int64)
        plate_img = get_rotated_img(img[plate[1]:plate[3], plate[0]:plate[2]])
        plate_img = get_cropped_img(plate_img)
        #plate_img_mod = get_mod_img(plate_img)
        letter_pred = letters_model(plate_img)
        letters = torch.Tensor.cpu(letter_pred.xyxy[0])
        final_img = cv2.rectangle(final_img, (plate[0], plate[1]), (plate[2], plate[3]), (255, 0, 0), 2)
        x_vals = []
        for letter in letters:
            if letter[4] < thresh:
                continue
            letter = np.array(letter, dtype = np.int64)
            x_vals.append(letter[2])
            letter_img = plate_img[letter[1]:letter[3], letter[0]:letter[2]]
            letter_img = cv2.cvtColor(letter_img, cv2.COLOR_RGB2GRAY)
            letter_img = cv2.resize(letter_img, (96, 96))
            letter_img = get_bin(letter_img)/255
            letter_img = torch.from_numpy(letter_img)
            letter_img = letter_img.type(torch.FloatTensor)
            class_pred = torch.Tensor.cpu(class_model(letter_img)).tolist()
            class_pred = dic[np.argmax(class_pred)]
            if (letter[0] < plate_img.shape[1]/2):
                if class_pred == 'ه' or class_pred == 'د':
                    class_pred = '5'
                elif class_pred == 'ا' or class_pred == 'م':
                    class_pred = '1'
            plate_reading.append(class_pred)
        plate_reading = [num for _, num in sorted(zip(x_vals, plate_reading))]
        #plate_reading.reverse()
        if plate_reading != []:
            all_plates.append([plate_reading, (plate[0], plate[1]), int((plate[3] - plate[1])/3)])
    final_img = Image.fromarray(final_img)
    fontpath = r"C:\Users\PC\Documents\projects\carPlates\font\arial.ttf"
    draw = ImageDraw.Draw(final_img)
    for plate in all_plates:
        font = ImageFont.truetype(fontpath, plate[2])
        if plate[0] != []:
            plate_num = put_letters_together(plate[0])
        draw.text(plate[1], plate_num, font = font, fill = (0, 255, 0))
    font = ImageFont.truetype(fontpath, 28)
    t2 = time.time()
    draw.text((0, 0),   f"Time for processing is {int((t2 - t1)*100000) / 100000} second", font = font)
    return np.array(final_img)

for path in test_dirs[3:]:
    i = len(os.listdir("predictions"))
    img_path = r"C:\Users\PC\OneDrive\Desktop\test16.jpg"
    img_path = test_dirs[3]
    img_path = path
    img = Image.open(img_path)
    img = img_pred(np.array(img))
    img = Image.fromarray(img)
    img.save(f"predictions/{i}.jpg")

In [13]:
img_path = test_dirs[10]
img = Image.open(img_path)
img = img_pred(np.array(img))
img = Image.fromarray(img)
img.show()

import cv2

vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    frame = img_pred(frame)
    # Display the resulting frame
    cv2.imshow('frame', frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

In [17]:
import cv2
import numpy as np
   
# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture(r"E:\DataSets\carPlatesDataset\raw_vids\VID_20220712_205303.mp4")
   
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video  file")
    
# Read until video is completed
while(cap.isOpened()):    
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        frame = img_pred(frame)
        cv2.imshow('Frame', frame)
   
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
   
  # Break the loop
    #else: 
        #print("yes")
        #break
        
# When everything done, release 
# the video capture object
cap.release()
   
# Closes all the frames
cv2.destroyAllWindows()